## Modelo Preditivo: XGBoost (eXtreme Gradient Boosting) na base logarítmica


O eXtreme Gradient Boosting é um algoritmo de aprendizado de máquina baseado no método de boosting, que combina múltiplos modelos simples, chamados de árvores de decisão, usado tanto para tarefas de regressão, previsão numerica, quanto para classificação. Seu funcionamento é baseado na combinação de múltiplas árvores de decisão de maneira sequencial, utilizando o método de boosting para ajustar gradualmente os erros das iterações anteriores.

### Instruções para uso local ou remoto (Google Colab ou VScode)

Aqui deixaremos brevemente um passo a passo para que você usuário seja capaz de executar o código localmente ou remotamente pelo seu google drive, podendo escolher a forma mais viável para seu uso e conhecimento.

##### Google Colab

1. Faça o upload do seu arquivo .ipynb para o Google Drive.
2. Abra o Google Colab em seu navegador.
3. Clique em "Arquivo" no menu superior e selecione "Abrir notebook".
4. Na guia "Upload", clique em "Procurar" e selecione o arquivo .ipynb que você enviou para o Google Drive.
5. Após selecionar o arquivo, clique em "Abrir".
6. Aguarde o carregamento do notebook no Google Colab.
7. Agora que você carregou o notebook no Google Colab, você pode fazer as alterações necessárias nos arquivos e caminhos para se adequar ao seu ambiente específico.

No notebook, a célula seguinte à essa contém as leituras dos arquivos CSV com o caminho do drive do criador desse notebook.
Comente as linhas que fazem referência aos arquivos locais e descomente as linhas que fazem referência ao Google Drive. Por exemplo:

- descomente as linhas que começam com # from google.colab import drive;
- comente as linhas que começam com tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv").

Certifique-se de que os arquivos CSV estejam localizados no diretório correto em seu ambiente virtual. Por exemplo, se você tiver uma pasta chamada "data" no mesmo diretório do notebook, coloque os arquivos CSV nessa pasta e ajuste seus nomes. Possivelmente os arquivos vão seguir o seguinte padrão, mesmo no seu drive:

tabela_Meta = pd.read_csv("/content/drive/MyDrive/NomeDaPastaDosArquivos/arquivo.csv)

Salve as alterações no notebook.
Agora você pode executar as células do notebook no Google Colab e as alterações nos arquivos e caminhos serão aplicadas ao seu ambiente do colab. Lembre-se de que você não precisará ter as bibliotecas necessárias instaladas em seu ambiente local para executar o código corretamente, dado que ao executar pela ferramenta do google essas dependências estarão aplicadas por padrão

##### VScode

1. Coloque os arquivos CSV dentro da pasta "data" desse notebook

No notebook, a célula seguinte à essa contém as leituras dos arquivos CSV com o caminho do drive do criador desse notebook.
Comente as linhas que fazem referência aos arquivos locais e descomente as linhas que fazem referência ao Google Drive. Por exemplo:

- descomente as linhas que começam com # from google.colab import drive;
- comente as linhas que começam com tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv").

Certifique-se de que os arquivos CSV estejam localizados no diretório correto em seu ambiente virtual. Por exemplo, se você tiver uma pasta chamada "data" no mesmo diretório do notebook, coloque os arquivos CSV nessa pasta e ajuste seus nomes. Possivelmente os arquivos vão seguir o seguinte padrão, mesmo no seu drive:

tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv")

Salve as alterações no notebook.
Agora você pode executar as células do notebook no VScode e as alterações nos arquivos e caminhos serão aplicadas ao seu ambiente do colab. Lembre-se de que você precisará ter as bibliotecas necessárias instaladas em seu ambiente local para executar o código corretamente:

Para fazer a instalação, basta abrir o terminal integrado e inserir o seguinte:

In [ ]:
#pip install pandas numpy matplotlib scikit-learn

### Execução do Modelo

O primeiro passo para realizarmos nossas previsões, é importar as bibliotecas que serão usadas.

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error


Abaixo, usei os arquivos na pasta "data" com os nomes descritos, mas lembre-se de mudar conforme a necessidade

In [40]:
# # pegando arquivos csv do drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Lendo os arquivos CSV
# tabela_Meta = pd.read_csv("/content/drive/MyDrive/Primeiro Ano/Módulo 3 - Modelo Preditivo Gazeta/Base de Dados Limpas/BASE INTELI_META-limpo.csv")
# tabela_Agosto = pd.read_csv("/content/drive/MyDrive/Primeiro Ano/Módulo 3 - Modelo Preditivo Gazeta/Base de Dados Limpas/tratada_BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv")

# para realizar o processo localmente, descomente as linhas abaixo e comente as linhas acima.
tabela_Meta = pd.read_csv("../data/dados_tratados/BASE INTELI_META-limpo.csv")
tabela_Agosto = pd.read_csv("../data/dados_tratados/tratada_BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv")

Com os arquivos em mão, vamos definir valores máximos no formato de exibição e agrupar colunas, visando somar as de valor númerico de acordo com as exigências do parceiro. No fim, a saída mostrará uma representação da tabela filtrada com as features usadas será mostrada.

In [ ]:
# Definindo formato de exibição
# pd.set_option('display.float_format', '{:.0f}'.format)
ipca_padrao = tabela_Agosto.groupby(['Ano', 'Mês'])['IPCA ES'].first().reset_index()


# Agrupar e somar 'Vl Liquido Final' e 'Outra Coluna'
tabela_agosto_segmento = tabela_Agosto.groupby(['Ano', 'Mês','Veiculo', 'Origem','Segmento']).agg({
    'Vl Liquido Final': 'sum',
    # 'IPCA ES': 'sum',
}).reset_index()

tabela_agosto_segmento = tabela_agosto_segmento.merge(ipca_padrao, on=['Ano', 'Mês'], how='left')


tabela_agosto_segmento.head()

Portanto, de acordo com a tabela filtrada, as features utilizadas são as:
- **Ano**: Permite avaliar o desempenho da receita ao longo do ano.
- **Mês**: Indica em qual mês a receita teve maior impacto.
- **Vl. Líquido**: Utilizado para prever a receita final, pois os ganhos estão relacionados a esse valor.
- **Veículo**: Analisa o impacto de cada veículo de venda na receita final.
- **Origem**: Identifica a localidade da venda para verificar a região de maior impacto na receita.
- **Segmento**: Permite filtrar receitas por setor econômico de forma mais detalhada (segmento).
- **% Ating. Meta**: Permite avaliar a eficiência em atingir as metas de receita estabelecidas.


Daqui em diante, iremos trabalhar mais com ajustes das colunas para que o modelo seja capaz de compreender as informações.

 Abaixo, as colunas categóricas (aquelas que contém texto) serão transformadas em numéricas, de modo a pouco comprometer o desempenho do modelo.

In [42]:
categorical_columns = ['Origem', 'Veiculo', 'Segmento']
X = tabela_agosto_segmento.drop('Vl Liquido Final', axis=1)
y = tabela_agosto_segmento['Vl Liquido Final']

# Aplicando OneHotEncoder nas colunas categóricas
onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)

# Transformar as colunas categóricas e converter para DataFrame
X_encoded = onehot_encoder.fit_transform(X[categorical_columns])

# Converter para DataFrame com os nomes das colunas correspondentes
X_encoded_df = pd.DataFrame(X_encoded, columns=onehot_encoder.get_feature_names_out(categorical_columns))

# Remover as colunas categóricas originais de X
X = X.drop(categorical_columns, axis=1)

# Concatenar as colunas codificadas com o restante de X
X = pd.concat([X.reset_index(drop=True), X_encoded_df.reset_index(drop=True)], axis=1)

Após discussões, o grupo decidiu testar o modelo aplicando uma normalização na coluna ``Vl Liquido Final`` utilizando a base logarítmica. A normalização foi realizada da seguinte forma:

In [43]:
y_log = np.log(y)

Após, faremos a divisão das amostras de treinamento e teste do modelo, e depois o treino do modelo com o grupo de treino, equivalente à 70% da base total

In [44]:
X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.3, random_state=42)

In [ ]:

xgb = XGBRegressor(
    colsample_bytree=1, 
    learning_rate=0.3, 
    max_depth=10,  
    min_child_weight=1, 
    n_estimators=int(73.87222023438979),  
    reg_alpha=3.9633679770424077, 
    reg_lambda=5, 
    subsample=1
)


xgb.fit(X_train, y_train_log)

Por fim, testaremos a previsão do modelo, usando os 30% restante dos dados, e voltamos os valores da coluna ``Vl Liquido Final`` a sua base antiga.

In [46]:
# Fazer previsões
y_pred_log = xgb.predict(X_test)

y_pred_exp = np.exp(y_pred_log)
y_test_exp = np.exp(y_test_log)

Nesse contexto, a fim de otimizar os hiperparâmetros que melhor se encaixam na situação, o grupo decidiu usar o algoritmo Grid Search, onde ele testa todas as combinações possíveis de hiperparâmetros em uma grade definida pelo usuário. A grade decidida pelo grupo tinha os seguintes hiperparâmetros

- n_estimators: Define o número de árvores no modelo, ou seja, quantas iterações o algoritmo realizará. Um valor mais alto pode melhorar a precisão, mas pode aumentar o tempo de treinamento e o risco de overfitting.
- max_depth: Define a profundidade máxima de cada árvore. Profundidades maiores permitem que o modelo capture padrões mais complexos, mas também aumentam o risco de overfitting.
- learning_rate Controla a taxa de aprendizado do modelo, ou seja, o quanto cada árvore contribui para o modelo final. Um valor mais baixo faz com que o modelo aprenda de maneira mais gradual, potencialmente melhorando a precisão, mas exigindo mais iterações.
- random_state: Garante a reprodutibilidade dos resultados, fixando a semente do gerador de números aleatórios. Isso permite que os mesmos dados e parâmetros produzam sempre o mesmo modelo.
- subsample: Define a fração dos dados usados em cada árvore. Um valor de 1 significa que todas as amostras serão utilizadas para construir cada árvore. Valores menores podem ajudar a reduzir o overfitting.
- min_child_weight: Determina o peso mínimo necessário para dividir um nó em uma árvore. Valores menores permitem a criação de nós com menos amostras, o que pode aumentar a flexibilidade, mas aumentar o risco de overfitting.
- colsample_bytree: Especifica a fração de recursos (features) que será usada em cada árvore. Um valor de 1 significa que todas as features serão usadas. Reduzir esse valor pode evitar o overfitting.
- reg_alpha: Controla a regularização L1, que adiciona uma penalidade ao modelo para evitar overfitting. Um valor de 0 significa que essa regularização não está sendo aplicada.
- reg_lambda: Controla a regularização L2, que também adiciona uma penalidade ao modelo, mas de maneira diferente da L1. Um valor maior impõe uma penalização mais forte, ajudando a prevenir overfitting.

Abaixo, está o código comentando do Grid Search, pois esse algoritmo demora bastante para rodar.

In [47]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'n_estimators': list(range(50, 80)),
#     'learning_rate': [0.01, 0.1, 0.2, 0.3],
#     'max_depth': [5, 6, 7, 8, 9, 10],
#     'subsample': [0.1, 0.3, 0.7, 1],
#     'min_child_weight': [0.5, 1, 2, 5],
#     'colsample_bytree': [0.7, 0.8, 1], 
#     'reg_alpha': [0, 0.01, 0.1, 1, 2, 3, 4],        
#     'reg_lambda': [1, 1.5, 2, 3, 4, 5]           
# }

# # Configurando o Grid Search com validação cruzada
# grid_search = GridSearchCV(
#     estimator=xgb,
#     param_grid=param_grid,
#     scoring='neg_mean_absolute_error',  # Métrica de avaliação
#     cv=3,                               # Número de folds para validação cruzada
#     verbose=1,
#     n_jobs=-1                           # Usa todos os núcleos disponíveis
# )

# # Ajustando o Grid Search ao conjunto de dados
# grid_search.fit(X_train, y_train_log)

# # Imprimindo os melhores parâmetros e o melhor resultado
# print("Melhores Parâmetros:", grid_search.best_params_)
# print("Melhor Score:", -grid_search.best_score_)

Dessa forma, com o Grid Search chegamos nos seguintes valores para os hiperparâmetros:
- colsample_bytree: 1,
- learning_rate: 0.3,
- max_depth: 10,
- min_child_weight: 1,
- n_estimators: 74,
- reg_alpha: 4,
- reg_lambda: 5,
- subsample: 1

Implementando esses hiperparâmetros no modelo deste notebook, descritos acima, alcançaremos as seguintes métricas:

In [ ]:
mse = mean_squared_error(y_test_exp, y_pred_exp)
print(f"Root Mean Squared Error: {sqrt(mse)}")
mae = mean_absolute_error(y_test_exp, y_pred_exp)
print(f"Mean Absolute Error: {mae}")
r2 = r2_score(y_test_log, y_pred_log)
print(f"R² Score: {r2*100:.2f}%")
mape = mean_absolute_percentage_error(y_test_exp, y_pred_exp)
print (f"MAPE: {mape*100:.2f}%")


### Análise do Modelo
Agora, veremos as métricas resultantes desse modelo, como MSE, MAE, R² e MAPE.

#### Erro Médio Absoluto (MAE - do inglês *Mean Absoluto Error*)

O MAE é uma métrica utilizada para avaliar a qualidade de modelos de regressão, calculando a média das diferenças absolutas entre os valores reais e os valores preditos. Quanto menor o valor do MAE, melhor o desempenho do modelo. Abaixo, apresentamos o cálculo desse indicador no modelo do grupo:<br>

In [ ]:
mae = mean_absolute_error(y_test_exp, y_pred_exp)
print(f"Mean Absolute Error: {mae}")

#### Raiz do Erro Quadrático Médio (RMSE - do inglês *Root Mean Squared Error*)

O RMSE (Root Mean Squared Error) é uma métrica de avaliação utilizada em modelos de regressão que, assim como o MSE (Mean Squared Error), mede a diferença entre os valores reais e os valores previstos. No RMSE, essa diferença é elevada ao quadrado, o que elimina os sinais negativos e dá mais peso a grandes erros. A diferença principal está no fato de que, ao final, é aplicada a raiz quadrada no valor resultante, tornando a métrica mais interpretável, pois os erros são trazidos de volta à mesma escala das variáveis originais. Portanto, quanto menor o RMSE, melhor o desempenho do modelo.

In [ ]:
rmse = mean_squared_error(y_test_exp, y_pred_exp)
print(f"Root Mean Squared Error: {sqrt(rmse)}")

#### Coeficiente de Determinação (R²)

O Coeficiente de Determinação R² é uma métrica que representa o percentual da variância dos dados previstos, ou seja, o quão explicativo é o modelo em relação aos dados de acordo com o quão distante esses valores estão do valor central (médio). Uma vez que a fórmula do R² considera a subtração desta conta por 1, quando menor o percentual obtido, melhor é a explicação do modelo. Este, no entanto, não é suficiente para ter uma noção geral da performace do modelo, dependendo de outras métricas (como o MAE e o MSE). Abaixo, apresentamos o cálculo desse indicador no modelo do grupo:

In [ ]:
r2 = r2_score(y_test_log, y_pred_log)
print(f"R² Score: {r2*100:.2f}%")

#### Erro Médio Absoluto Percentual (MAPE - do inglês Mean Absolute Percentage Error)

O MAPE é uma métrica utilizada para avaliar a precisão de modelos de regressão, calculando a média dos erros percentuais absolutos entre os valores reais e os valores preditos. Quanto menor o valor do MAPE, melhor o desempenho do modelo. Abaixo, apresentamos o cálculo desse indicador no modelo do grupo:

In [ ]:
mape = mean_absolute_percentage_error(y_test_exp, y_pred_exp)
print (f"MAPE: {mape*100:.2f}%")

#### Conclusões a partir das métricas avaliadas

O modelo de XGBoost, com normalização dos dados na base logaritmica, está funcionando muito bem com valores que variam de milhares a milhões. O R2 Score de 73,16% indica que o modelo explica quase toda a variabilidade dos dados. Além disso, o modelo faz previsões com precisão mais que suficiente para a maioria das aplicações práticas, porque o MAE, MSE, e MAPE são baixos em relação à escala dos dados fornecidos. Quando comparado com outros modelos, como Random Forest e XGBoost sem normalização, o modelo atual se mostrou menos propício ao overfitting.
Esses níveis de erro mostram que o modelo está bem calibrado, proporcionando resultados confiáveis, uma vez que estão adequados para valores de grande magnitude como Valor Líquido da Rede Gazeta.

Por fim, teremos a representação de uma figura relevante para a compreensão do desempenho do modelo, um gráfico onde mostra uma comparação de valores reais e as previsões do modelo. Onde, a linha roxa representa as previsões do modelo, enquanto os pontos vermelhor representam os valores reais que temos na tabela.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test_exp, y_pred_exp, color='red', alpha=0.6)
plt.plot([y_test_exp.min(), y_test_exp.max()], [y_test_exp.min(), y_test_exp.max()], color='purple', lw=3)  # Linha de identidade
plt.xlabel('Valores Reais')
plt.ylabel('Previsões')
plt.title('Comparação entre Valores Reais e Previsões - Regressão Polinomial')
plt.grid(True)
plt.show()